# Minify DB

In [1]:
import sqlite3

In [2]:
# CHANGE TO_MINIFY_DATABASE_FILENAME to a copy of a stage database. This database will be modified!
TO_MINIFY_DATABASE_FILENAME = 'word_cefr_stage_to_minify.db'
OUTPUT_DATABASE_FILENAME = 'word_cefr_minified.db'

In [3]:
def create_tables(cursor: sqlite3.Cursor, conn: sqlite3.Connection):
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS words (
        word_id INTEGER PRIMARY KEY AUTOINCREMENT,
        word VARCHAR(25) UNIQUE NOT NULL,
        stem_word_id INTEGER,

        FOREIGN KEY (stem_word_id) REFERENCES words (word_id)
    )
    ''')

    cursor.execute('''
    CREATE INDEX IF NOT EXISTS idx_word ON words (word)
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS pos_tags (
            tag_id INTEGER PRIMARY KEY AUTOINCREMENT,
            tag VARCHAR(4) UNIQUE NOT NULL,
            description VARCHAR(40) NOT NULL
        )
    ''')

    cursor.execute('''
    CREATE INDEX IF NOT EXISTS idx_pos_tag ON pos_tags (tag)
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS word_pos (
        word_pos_id INTEGER PRIMARY KEY AUTOINCREMENT,
        word_id INTEGER NOT NULL,
        pos_tag_id INTEGER NOT NULL,
        lemma_word_id INTEGER,
        frequency_count INTEGER NOT NULL,
        level REAL NOT NULL,

        UNIQUE (word_id, pos_tag_id),
        FOREIGN KEY (word_id) REFERENCES words(word_id),
        FOREIGN KEY (pos_tag_id) REFERENCES pos_tags(tag_id),
        FOREIGN KEY (lemma_word_id) REFERENCES words(word_id)
    )
    ''')

    cursor.execute('''
    CREATE INDEX IF NOT EXISTS idx_word_pos_lemma ON word_pos (lemma_word_id)
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS categories (
        category_id INTEGER PRIMARY KEY AUTOINCREMENT,
        category_title VARCHAR(255) NOT NULL
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS word_categories (
        word_pos_id INTEGER NOT NULL,
        category_id INTEGER NOT NULL,

        UNIQUE (word_pos_id, category_id),
        FOREIGN KEY (word_pos_id) REFERENCES word_pos(word_pos_id),
        FOREIGN KEY (category_id) REFERENCES categories(category_id)
    )
    ''')

    cursor.execute('''
    CREATE INDEX IF NOT EXISTS idx_word_categories_word_pos_id ON word_categories (word_pos_id)
    ''')

    conn.commit()

In [4]:
conn = sqlite3.connect(TO_MINIFY_DATABASE_FILENAME)
cursor = conn.cursor()

In [5]:
cursor.execute('''
    ALTER TABLE word_pos
    ADD COLUMN frequency_count INTEGER
''')

cursor.execute('''
    ALTER TABLE word_pos
    ADD COLUMN level REAL
''')

conn.commit()

In [6]:
cursor.execute('''
    UPDATE word_pos
    SET frequency_count = COALESCE(
        (SELECT SUM(frequency) FROM year_frequency WHERE year_frequency.word_pos_id = word_pos.word_pos_id),
        10000
    )
''')

cursor.execute('''
    UPDATE word_pos
    SET level = (
        SELECT AVG(level)
        FROM word_pos_levels
        WHERE word_pos_levels.word_pos_id = word_pos.word_pos_id
    )
''')

conn.commit()

In [7]:
conn_main = sqlite3.connect(OUTPUT_DATABASE_FILENAME)
cursor_main = conn_main.cursor()

create_tables(cursor_main, conn_main)
conn_main.commit()

In [8]:
cursor_main.execute('ATTACH DATABASE ? AS source_db', (TO_MINIFY_DATABASE_FILENAME,))

cursor_main.execute('INSERT INTO main.words SELECT * FROM source_db.words')
cursor_main.execute('INSERT INTO main.pos_tags SELECT * FROM source_db.pos_tags')
cursor_main.execute('INSERT INTO main.word_pos SELECT * FROM source_db.word_pos')
cursor_main.execute('INSERT INTO main.categories SELECT * FROM source_db.categories')
cursor_main.execute('INSERT INTO main.word_categories SELECT * FROM source_db.word_categories')

conn_main.commit()

cursor_main.execute('DETACH DATABASE source_db')
conn_main.commit()

In [ ]:
conn.close()
conn_main.close()